# DSPy Optimizer Benchmark

This notebook benchmarks all available DSPy optimizers against the AI text detection task from `dspy-in-8-steps.ipynb`.

**Metrics tracked for each optimizer:**
- API calls (number of LM calls)
- Total tokens used (prompt + completion)
- Cost estimate (based on OpenAI pricing)
- Accuracy uplift vs baseline
- Time taken to optimize

**Models used:**
- Task model: `openai/gpt-5-mini`
- Teacher/reflection model: `openai/gpt-5`
- Fine-tuning model: `Qwen3-1.7B` (for GRPO, BootstrapFinetune, BetterTogether)

In [1]:
# Install dependencies
!uv pip install dspy pandas python-dotenv gepa -q

In [2]:
import dspy
import pandas as pd
import time
import os
import random
import copy
import threading
import contextlib
import signal
import sys
import time as time_module
from concurrent.futures import FIRST_COMPLETED, ThreadPoolExecutor, wait
import tqdm
import logging
from dotenv import load_dotenv
from dataclasses import dataclass, field
from typing import Any

# Load environment variables
load_dotenv()

# Pricing per 1M tokens (as of Dec 2025)
# Source: https://openai.com/api/pricing/
PRICING = {
    "openai/gpt-5-mini": {"input": 0.25, "output": 2.00},
    "openai/gpt-5": {"input": 1.25, "output": 10.00},
}

NUM_THREADS = 16

def calculate_cost(usage: dict) -> float:
    """Calculate cost from usage dict."""
    total_cost = 0.0
    for model, data in usage.items():
        if model in PRICING:
            prompt_tokens = data.get("prompt_tokens", 0)
            completion_tokens = data.get("completion_tokens", 0)
            total_cost += (prompt_tokens / 1_000_000) * PRICING[model]["input"]
            total_cost += (completion_tokens / 1_000_000) * PRICING[model]["output"]
    return total_cost



In [3]:

# =============================================================================
# Monkey-patch for DSPy parallelizer bug (affects dspy.track_usage())
# 
# Bug: Line 95 in parallelizer.py tries to access thread_local_overrides.overrides
# which doesn't exist (ContextVar has no .overrides attribute). This silently
# breaks context propagation to worker threads.
#
# Fix: Build the complete overrides dict BEFORE calling set(), and don't deep
# copy the usage_tracker so it's shared across threads.
# =============================================================================
from dspy.utils.parallelizer import ParallelExecutor
from dspy.dsp.utils.settings import thread_local_overrides

_logger = logging.getLogger(__name__)

def _fixed_execute_parallel(self, function, data):
    """Fixed version of ParallelExecutor._execute_parallel."""
    results = [None] * len(data)
    job_cancelled = "cancelled"
    start_time_map = {}
    start_time_lock = threading.Lock()
    resubmitted = set()
    
    parent_overrides = thread_local_overrides.get().copy()

    def worker(parent_ovr, submission_id, index, item):
        if self.cancel_jobs.is_set():
            return index, job_cancelled
        with start_time_lock:
            start_time_map[submission_id] = time_module.time()

        original = thread_local_overrides.get()
        # FIX: Build complete overrides dict BEFORE calling set()
        # Don't deep copy usage_tracker - share it across threads for aggregation
        new_overrides = {**original, **parent_ovr}
        token = thread_local_overrides.set(new_overrides)

        try:
            return index, function(item)
        finally:
            thread_local_overrides.reset(token)

    @contextlib.contextmanager
    def interrupt_manager():
        if threading.current_thread() is threading.main_thread():
            orig_handler = signal.getsignal(signal.SIGINT)
            def handler(sig, frame):
                self.cancel_jobs.set()
                _logger.warning("SIGINT received. Cancelling.")
                orig_handler(sig, frame)
            signal.signal(signal.SIGINT, handler)
            try:
                yield
            finally:
                signal.signal(signal.SIGINT, orig_handler)
        else:
            yield

    executor = ThreadPoolExecutor(max_workers=self.num_threads)
    try:
        with interrupt_manager():
            futures_map = {}
            futures_set = set()
            submission_counter = 0

            for idx, item in enumerate(data):
                f = executor.submit(worker, parent_overrides, submission_counter, idx, item)
                futures_map[f] = (submission_counter, idx, item)
                futures_set.add(f)
                submission_counter += 1

            pbar = tqdm.tqdm(
                total=len(data),
                dynamic_ncols=True,
                disable=self.disable_progress_bar,
                file=sys.stdout,
            )

            def all_done():
                return all(r is not None for r in results)

            while futures_set and not self.cancel_jobs.is_set():
                if all_done():
                    break
                done, not_done = wait(futures_set, timeout=1, return_when=FIRST_COMPLETED)
                for f in done:
                    futures_set.remove(f)
                    try:
                        index, outcome = f.result()
                    except Exception:
                        pass
                    else:
                        if outcome != job_cancelled and results[index] is None:
                            if isinstance(outcome, Exception):
                                with self.error_lock:
                                    self.failed_indices.append(index)
                                    self.exceptions_map[index] = outcome
                                results[index] = None
                            else:
                                results[index] = outcome

                        if self.compare_results:
                            vals = [r[-1] for r in results if r is not None]
                            self._update_progress(pbar, sum(vals), len(vals))
                        else:
                            self._update_progress(pbar, len([r for r in results if r is not None]), len(data))

                if all_done():
                    break

                if 0 < self.timeout and len(not_done) <= self.straggler_limit:
                    now = time_module.time()
                    for f in list(not_done):
                        if f not in resubmitted:
                            sid, idx, item = futures_map[f]
                            with start_time_lock:
                                st = start_time_map.get(sid, None)
                            if st and (now - st) >= self.timeout:
                                resubmitted.add(f)
                                nf = executor.submit(worker, parent_overrides, submission_counter, idx, item)
                                futures_map[nf] = (submission_counter, idx, item)
                                futures_set.add(nf)
                                submission_counter += 1

            pbar.close()

    finally:
        executor.shutdown(wait=False)

    if self.cancel_jobs.is_set():
        _logger.warning("Execution cancelled due to errors or interruption.")
        raise Exception("Execution cancelled due to errors or interruption.")

    return results

# Apply the fix
ParallelExecutor._execute_parallel = _fixed_execute_parallel
print("Applied DSPy parallelizer fix for usage tracking")

Applied DSPy parallelizer fix for usage tracking


In [4]:
# Initialize models
task_lm = dspy.LM(
    "openai/gpt-5-mini",
    api_key=os.getenv("OPENAI_API_KEY"),
    temperature=1.0,
    max_tokens=16000,
    cache=False # In order to track latency and cost
)

teacher_lm = dspy.LM(
    "openai/gpt-5",  # Teacher/reflection model
    api_key=os.getenv("OPENAI_API_KEY"),
    temperature=1.0,
    max_tokens=16000,
    cache=False # In order to track latency and cost
)

# Configure default LM
dspy.configure(lm=task_lm)

In [5]:
# Load dataset (same as dspy-in-8-steps.ipynb)
csv_path = 'ai_vs_human200.csv'
df = pd.read_csv(csv_path)
examples = df.to_dict(orient='records')

# Convert to DSPy Examples
dataset = [
    dspy.Example(**ex).with_inputs("text")
    for ex in examples
]

# Reproducible shuffle and split
random.seed(42)
random.shuffle(dataset)

trainset = dataset[:len(dataset)//2]
valset = dataset[len(dataset)//2:]

print(f"Loaded: {csv_path}")
print(f"Training set: {len(trainset)} examples")
print(f"Validation set: {len(valset)} examples")

Loaded: ai_vs_human200.csv
Training set: 101 examples
Validation set: 101 examples


In [6]:
# Define Signature and Module (same as dspy-in-8-steps.ipynb)
class DetectAIText(dspy.Signature):
    """Detects whether text is AI-generated."""
    text: str = dspy.InputField(description="The text to analyze")
    is_ai: bool = dspy.OutputField(description="Whether the text is AI-generated")


class AIDetector(dspy.Module):
    def __init__(self):
        super().__init__()
        # Chain of thought reasoning
        self.detect = dspy.ChainOfThought(DetectAIText)

    def forward(self, text: str):
        return self.detect(text=text)

In [7]:
# Define metrics (same as dspy-in-8-steps.ipynb)

# Standard exact match metric
def exact_match(example, response, trace=None):
    return 1 if example.is_ai == response.is_ai else 0


# Exact match metric with feedback for GEPA
def exact_match_with_feedback(example, response, trace=None, pred_name=None, pred_trace=None):
    score = 1 if example.is_ai == response.is_ai else 0
    if pred_name:
        return dspy.Prediction(score=score, feedback=example.notes)
    else:
        return score

In [8]:
@dataclass
class BenchmarkResult:
    """Results from benchmarking an optimizer."""
    optimizer_name: str
    baseline_accuracy: float
    optimized_accuracy: float
    accuracy_uplift: float
    total_tokens: int
    prompt_tokens: int
    completion_tokens: int
    cost_usd: float
    time_seconds: float
    notes: str = ""
    usage_by_model: dict = field(default_factory=dict)


def run_baseline_evaluation(module_class, devset, metric):
    """Run baseline evaluation and return accuracy."""
    detector = module_class()
    evaluator = dspy.Evaluate(
        devset=devset,
        metric=metric,
        num_threads=NUM_THREADS,
        display_progress=True,
        display_table=False,
    )
    result = evaluator(detector)
    return result.score / 100  # Convert percentage to decimal


def evaluate_optimized(optimized_module, devset, metric):
    """Evaluate optimized module and return accuracy."""
    evaluator = dspy.Evaluate(
        devset=devset,
        metric=metric,
        num_threads=NUM_THREADS,
        display_progress=True,
        display_table=False,
    )
    result = evaluator(optimized_module)
    return result.score / 100


def print_benchmark_result(result: BenchmarkResult):
    """Print benchmark results in a standard format."""
    print("\n" + "="*60)
    print(f"OPTIMIZER: {result.optimizer_name}")
    print("="*60)
    print(f"Baseline Accuracy:   {result.baseline_accuracy*100:.1f}%")
    print(f"Optimized Accuracy:  {result.optimized_accuracy*100:.1f}%")
    print(f"Accuracy Uplift:     {result.accuracy_uplift*100:+.1f}%")
    print("-"*60)
    print(f"Total Tokens:        {result.total_tokens:,}")
    print(f"  - Prompt:          {result.prompt_tokens:,}")
    print(f"  - Completion:      {result.completion_tokens:,}")
    print(f"Estimated Cost:      ${result.cost_usd:.4f}")
    print(f"Time Taken:          {result.time_seconds:.1f}s")
    if result.notes:
        print(f"Notes:               {result.notes}")
    print("="*60 + "\n")


# Store all results for final comparison
all_results: list[BenchmarkResult] = []

## Baseline Evaluation

First, let's establish our baseline accuracy without any optimization.

In [9]:
# Run baseline evaluation
print("Running baseline evaluation...")
baseline_accuracy = run_baseline_evaluation(AIDetector, valset, exact_match)

print(f"\nBaseline accuracy: {baseline_accuracy*100:.1f}%")

Running baseline evaluation...
Average Metric: 66.00 / 101 (65.3%): 100%|██████████| 101/101 [00:55<00:00,  1.82it/s]

2025/12/04 12:15:52 INFO dspy.evaluate.evaluate: Average Metric: 66 / 101 (65.3%)




Baseline accuracy: 65.3%


---
## 1. LabeledFewShot

The simplest optimizer - just adds labeled examples as few-shot demonstrations. No metric needed.

In [30]:
from dspy.teleprompt import LabeledFewShot

print("Running LabeledFewShot optimizer...")

start_time = time.time()
with dspy.track_usage() as usage:
    optimizer = LabeledFewShot(k=4)
    optimized_detector = optimizer.compile(
        AIDetector(),
        trainset=trainset
    )
    
    # Evaluate
    optimized_accuracy = evaluate_optimized(optimized_detector, valset, exact_match)

elapsed = time.time() - start_time
total_usage = usage.get_total_tokens()

# Calculate totals
total_tokens = sum(d.get("total_tokens", 0) for d in total_usage.values())
prompt_tokens = sum(d.get("prompt_tokens", 0) for d in total_usage.values())
completion_tokens = sum(d.get("completion_tokens", 0) for d in total_usage.values())

result = BenchmarkResult(
    optimizer_name="LabeledFewShot",
    baseline_accuracy=baseline_accuracy,
    optimized_accuracy=optimized_accuracy,
    accuracy_uplift=optimized_accuracy - baseline_accuracy,
    total_tokens=total_tokens,
    prompt_tokens=prompt_tokens,
    completion_tokens=completion_tokens,
    cost_usd=calculate_cost(total_usage),
    time_seconds=elapsed,
    notes="No metric required. Simply adds k random examples as demos.",
    usage_by_model=total_usage,
)

print_benchmark_result(result)
all_results.append(result)

Running LabeledFewShot optimizer...
Average Metric: 87.00 / 101 (86.1%): 100%|██████████| 101/101 [00:49<00:00,  2.05it/s]

2025/12/03 15:41:48 INFO dspy.evaluate.evaluate: Average Metric: 87 / 101 (86.1%)




OPTIMIZER: LabeledFewShot
Baseline Accuracy:   72.3%
Optimized Accuracy:  86.1%
Accuracy Uplift:     +13.9%
------------------------------------------------------------
Total Tokens:        94,497
  - Prompt:          54,814
  - Completion:      39,683
Estimated Cost:      $0.0931
Time Taken:          49.3s
Notes:               No metric required. Simply adds k random examples as demos.



---
## 2. BootstrapFewShot

Bootstraps demonstrations by running the teacher on training examples and selecting successful traces.

In [31]:
from dspy.teleprompt import BootstrapFewShot

print("Running BootstrapFewShot optimizer...")

start_time = time.time()
with dspy.track_usage() as usage:
    optimizer = BootstrapFewShot(
        metric=exact_match,
        max_bootstrapped_demos=4,
        max_labeled_demos=4,
        max_rounds=1,
    )
    optimized_detector = optimizer.compile(
        AIDetector(),
        trainset=trainset,
    )
    
    # Evaluate
    optimized_accuracy = evaluate_optimized(optimized_detector, valset, exact_match)

elapsed = time.time() - start_time
total_usage = usage.get_total_tokens()

total_tokens = sum(d.get("total_tokens", 0) for d in total_usage.values())
prompt_tokens = sum(d.get("prompt_tokens", 0) for d in total_usage.values())
completion_tokens = sum(d.get("completion_tokens", 0) for d in total_usage.values())

result = BenchmarkResult(
    optimizer_name="BootstrapFewShot",
    baseline_accuracy=baseline_accuracy,
    optimized_accuracy=optimized_accuracy,
    accuracy_uplift=optimized_accuracy - baseline_accuracy,
    total_tokens=total_tokens,
    prompt_tokens=prompt_tokens,
    completion_tokens=completion_tokens,
    cost_usd=calculate_cost(total_usage),
    time_seconds=elapsed,
    notes="Bootstraps successful traces as demonstrations.",
    usage_by_model=total_usage,
)

print_benchmark_result(result)
all_results.append(result)

Running BootstrapFewShot optimizer...


  4%|▍         | 4/101 [00:27<11:01,  6.82s/it]


Bootstrapped 4 full traces after 4 examples for up to 1 rounds, amounting to 4 attempts.
Average Metric: 97.00 / 101 (96.0%): 100%|██████████| 101/101 [00:42<00:00,  2.39it/s]

2025/12/03 15:44:26 INFO dspy.evaluate.evaluate: Average Metric: 97 / 101 (96.0%)




OPTIMIZER: BootstrapFewShot
Baseline Accuracy:   72.3%
Optimized Accuracy:  96.0%
Accuracy Uplift:     +23.8%
------------------------------------------------------------
Total Tokens:        117,456
  - Prompt:          79,121
  - Completion:      38,335
Estimated Cost:      $0.0965
Time Taken:          69.7s
Notes:               Bootstraps successful traces as demonstrations.



---
## 3. BootstrapFewShotWithRandomSearch

Tries multiple random configurations of bootstrapped demos and selects the best.

In [32]:
from dspy.teleprompt import BootstrapFewShotWithRandomSearch

print("Running BootstrapFewShotWithRandomSearch optimizer...")

start_time = time.time()
with dspy.track_usage() as usage:
    optimizer = BootstrapFewShotWithRandomSearch(
        metric=exact_match,
        max_bootstrapped_demos=4,
        max_labeled_demos=4,
        max_rounds=1,
        num_candidate_programs=8,  # Reduced for faster benchmarking
        num_threads=NUM_THREADS,
    )
    optimized_detector = optimizer.compile(
        AIDetector(),
        trainset=trainset,
        valset=valset,
    )
    
    # Evaluate
    optimized_accuracy = evaluate_optimized(optimized_detector, valset, exact_match)

elapsed = time.time() - start_time
total_usage = usage.get_total_tokens()

total_tokens = sum(d.get("total_tokens", 0) for d in total_usage.values())
prompt_tokens = sum(d.get("prompt_tokens", 0) for d in total_usage.values())
completion_tokens = sum(d.get("completion_tokens", 0) for d in total_usage.values())

result = BenchmarkResult(
    optimizer_name="BootstrapFewShotWithRandomSearch",
    baseline_accuracy=baseline_accuracy,
    optimized_accuracy=optimized_accuracy,
    accuracy_uplift=optimized_accuracy - baseline_accuracy,
    total_tokens=total_tokens,
    prompt_tokens=prompt_tokens,
    completion_tokens=completion_tokens,
    cost_usd=calculate_cost(total_usage),
    time_seconds=elapsed,
    notes="Random search over demo configurations.",
    usage_by_model=total_usage,
)

print_benchmark_result(result)
all_results.append(result)

Running BootstrapFewShotWithRandomSearch optimizer...
Going to sample between 1 and 4 traces per predictor.
Will attempt to bootstrap 8 candidate sets.
Average Metric: 68.00 / 101 (67.3%): 100%|██████████| 101/101 [00:54<00:00,  1.85it/s]

2025/12/03 15:48:38 INFO dspy.evaluate.evaluate: Average Metric: 68 / 101 (67.3%)



New best score: 67.33 for seed -3
Scores so far: [67.33]
Best score so far: 67.33
Average Metric: 83.00 / 101 (82.2%): 100%|██████████| 101/101 [00:51<00:00,  1.94it/s]

2025/12/03 15:49:31 INFO dspy.evaluate.evaluate: Average Metric: 83 / 101 (82.2%)



New best score: 82.18 for seed -2
Scores so far: [67.33, 82.18]
Best score so far: 82.18


...

Bootstrapped 3 full traces after 3 examples for up to 1 rounds, amounting to 3 attempts.
Average Metric: 99.00 / 101 (98.0%): 100%|██████████| 101/101 [00:39<00:00,  2.54it/s]

2025/12/03 15:57:23 INFO dspy.evaluate.evaluate: Average Metric: 99 / 101 (98.0%)



Scores so far: [67.33, 82.18, 97.03, 99.01, 99.01, 95.05, 99.01, 99.01, 96.04, 98.02, 98.02]
Best score so far: 99.01
11 candidate programs found.
Average Metric: 98.00 / 101 (97.0%): 100%|██████████| 101/101 [00:38<00:00,  2.65it/s]

2025/12/03 15:58:01 INFO dspy.evaluate.evaluate: Average Metric: 98 / 101 (97.0%)




OPTIMIZER: BootstrapFewShotWithRandomSearch
Baseline Accuracy:   72.3%
Optimized Accuracy:  97.0%
Accuracy Uplift:     +24.8%
------------------------------------------------------------
Total Tokens:        1,189,910
  - Prompt:          752,282
  - Completion:      437,628
Estimated Cost:      $1.0633
Time Taken:          617.3s
Notes:               Random search over demo configurations.



---
## 4. COPRO

Optimizes instructions using an iterative refinement approach.

In [33]:
from dspy.teleprompt import COPRO

print("Running COPRO optimizer...")

start_time = time.time()
with dspy.track_usage() as usage:
    optimizer = COPRO(
        prompt_model=teacher_lm,
        metric=exact_match,
        breadth=5,  # Number of candidates per iteration
        depth=2,    # Number of refinement iterations
        init_temperature=1.0,
        track_stats=True,
    )
    optimized_detector = optimizer.compile(
        AIDetector(),
        trainset=trainset,
        eval_kwargs={"num_threads": NUM_THREADS, "display_progress": True, "display_table": False},
    )
    
    # Evaluate
    optimized_accuracy = evaluate_optimized(optimized_detector, valset, exact_match)

elapsed = time.time() - start_time
total_usage = usage.get_total_tokens()

total_tokens = sum(d.get("total_tokens", 0) for d in total_usage.values())
prompt_tokens = sum(d.get("prompt_tokens", 0) for d in total_usage.values())
completion_tokens = sum(d.get("completion_tokens", 0) for d in total_usage.values())

result = BenchmarkResult(
    optimizer_name="COPRO",
    baseline_accuracy=baseline_accuracy,
    optimized_accuracy=optimized_accuracy,
    accuracy_uplift=optimized_accuracy - baseline_accuracy,
    total_tokens=total_tokens,
    prompt_tokens=prompt_tokens,
    completion_tokens=completion_tokens,
    cost_usd=calculate_cost(total_usage),
    time_seconds=elapsed,
    notes=f"Uses gpt-5 for instruction generation. depth={optimizer.depth}, breadth={optimizer.breadth}",
    usage_by_model=total_usage,
)

print_benchmark_result(result)
all_results.append(result)

Running COPRO optimizer...


2025/12/03 16:10:45 INFO dspy.teleprompt.copro_optimizer: Iteration Depth: 1/2.
2025/12/03 16:10:45 INFO dspy.teleprompt.copro_optimizer: At Depth 1/2, Evaluating Prompt Candidate #1/5 for Predictor 1 of 1.






[2025-12-03T16:10:45.526022]

System message:

Your input fields are:
1. `basic_instruction` (str): The initial instructions before optimization
Your output fields are:
1. `proposed_instruction` (str): The improved instructions for the language model
2. `proposed_prefix_for_output_field` (str): The string at the end of the prompt, which will help the model start solving the task
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## basic_instruction ## ]]
{basic_instruction}

[[ ## proposed_instruction ## ]]
{proposed_instruction}

[[ ## proposed_prefix_for_output_field ## ]]
{proposed_prefix_for_output_field}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        You are an instruction optimizer for large language models. I will give you a ``signature`` of fields (inputs and outputs) in English. Your task is to propose an instruction that will lead a good language model to perform the task well. Don't be 

2025/12/03 16:14:34 INFO dspy.evaluate.evaluate: Average Metric: 56 / 101 (55.4%)
2025/12/03 16:14:34 INFO dspy.teleprompt.copro_optimizer: At Depth 1/2, Evaluating Prompt Candidate #2/5 for Predictor 1 of 1.







[2025-12-03T16:10:45.526022]

System message:

Your input fields are:
1. `basic_instruction` (str): The initial instructions before optimization
Your output fields are:
1. `proposed_instruction` (str): The improved instructions for the language model
2. `proposed_prefix_for_output_field` (str): The string at the end of the prompt, which will help the model start solving the task
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## basic_instruction ## ]]
{basic_instruction}

[[ ## proposed_instruction ## ]]
{proposed_instruction}

[[ ## proposed_prefix_for_output_field ## ]]
{proposed_prefix_for_output_field}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        You are an instruction optimizer for large language models. I will give you a ``signature`` of fields (inputs and outputs) in English. Your task is to propose an instruction that will lead a good language model to perform the task well. Don't be

...






[2025-12-03T16:30:26.440183]

System message:

Your input fields are:
1. `attempted_instructions` (str):
Your output fields are:
1. `proposed_instruction` (str): The improved instructions for the language model
2. `proposed_prefix_for_output_field` (str): The string at the end of the prompt, which will help the model start solving the task
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## attempted_instructions ## ]]
{attempted_instructions}

[[ ## proposed_instruction ## ]]
{proposed_instruction}

[[ ## proposed_prefix_for_output_field ## ]]
{proposed_prefix_for_output_field}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        You are an instruction optimizer for large language models. I will give some task instructions I've tried, along with their corresponding validation scores. The instructions are arranged in increasing order based on their scores, where higher scores indicate better quality.


2025/12/03 16:43:05 INFO dspy.evaluate.evaluate: Average Metric: 49 / 101 (48.5%)







[2025-12-03T16:30:26.440183]

System message:

Your input fields are:
1. `attempted_instructions` (str):
Your output fields are:
1. `proposed_instruction` (str): The improved instructions for the language model
2. `proposed_prefix_for_output_field` (str): The string at the end of the prompt, which will help the model start solving the task
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## attempted_instructions ## ]]
{attempted_instructions}

[[ ## proposed_instruction ## ]]
{proposed_instruction}

[[ ## proposed_prefix_for_output_field ## ]]
{proposed_prefix_for_output_field}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        You are an instruction optimizer for large language models. I will give some task instructions I've tried, along with their corresponding validation scores. The instructions are arranged in increasing order based on their scores, where higher scores indicate better quality.


2025/12/03 16:44:45 INFO dspy.evaluate.evaluate: Average Metric: 101 / 101 (100.0%)




OPTIMIZER: COPRO
Baseline Accuracy:   72.3%
Optimized Accuracy:  100.0%
Accuracy Uplift:     +27.7%
------------------------------------------------------------
Total Tokens:        3,136,535
  - Prompt:          1,157,672
  - Completion:      1,978,863
Estimated Cost:      $4.4435
Time Taken:          2119.4s
Notes:               Uses gpt-4o for instruction generation. depth=2, breadth=5



---
## 5. MIPROv2

Combines instruction optimization with few-shot demo selection using Bayesian optimization (Optuna).

In [34]:
from dspy.teleprompt import MIPROv2

print("Running MIPROv2 optimizer...")

start_time = time.time()
with dspy.track_usage() as usage:
    optimizer = MIPROv2(
        metric=exact_match,
        prompt_model=teacher_lm,
        task_model=task_lm,
        auto="light",  # Use light preset for faster benchmarking
        num_threads=NUM_THREADS,
        max_bootstrapped_demos=4,
        max_labeled_demos=4,
        track_stats=True,
    )
    optimized_detector = optimizer.compile(
        AIDetector(),
        trainset=trainset,
        valset=valset,
    )
    
    # Evaluate
    optimized_accuracy = evaluate_optimized(optimized_detector, valset, exact_match)

elapsed = time.time() - start_time
total_usage = usage.get_total_tokens()

total_tokens = sum(d.get("total_tokens", 0) for d in total_usage.values())
prompt_tokens = sum(d.get("prompt_tokens", 0) for d in total_usage.values())
completion_tokens = sum(d.get("completion_tokens", 0) for d in total_usage.values())

result = BenchmarkResult(
    optimizer_name="MIPROv2",
    baseline_accuracy=baseline_accuracy,
    optimized_accuracy=optimized_accuracy,
    accuracy_uplift=optimized_accuracy - baseline_accuracy,
    total_tokens=total_tokens,
    prompt_tokens=prompt_tokens,
    completion_tokens=completion_tokens,
    cost_usd=calculate_cost(total_usage),
    time_seconds=elapsed,
    notes="Uses gpt-4o for instruction generation. auto='light' preset.",
    usage_by_model=total_usage,
)

print_benchmark_result(result)
all_results.append(result)

2025/12/03 16:46:24 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING LIGHT AUTO RUN SETTINGS:
num_trials: 10
minibatch: True
num_fewshot_candidates: 6
num_instruct_candidates: 3
valset size: 100

2025/12/03 16:46:24 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/12/03 16:46:24 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/12/03 16:46:24 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=6 sets of demonstrations...


Running MIPROv2 optimizer...
Bootstrapping set 1/6
Bootstrapping set 2/6
Bootstrapping set 3/6


  5%|▍         | 5/101 [00:34<11:01,  6.89s/it]


Bootstrapped 4 full traces after 5 examples for up to 1 rounds, amounting to 5 attempts.
Bootstrapping set 4/6


  1%|          | 1/101 [00:07<11:48,  7.09s/it]


...

2025/12/03 17:08:36 INFO dspy.evaluate.evaluate: Average Metric: 98 / 100 (98.0%)
2025/12/03 17:08:36 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [67.0, 98.0, 98.0]


2025/12/03 17:08:36 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 98.0
2025/12/03 17:08:36 INFO dspy.teleprompt.mipro_optimizer_v2: =======================
2025/12/03 17:08:36 INFO dspy.teleprompt.mipro_optimizer_v2: 

2025/12/03 17:08:36 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 98.0!


Average Metric: 100.00 / 101 (99.0%): 100%|██████████| 101/101 [01:52<00:00,  1.11s/it]

2025/12/03 17:10:29 INFO dspy.evaluate.evaluate: Average Metric: 100 / 101 (99.0%)




OPTIMIZER: MIPROv2
Baseline Accuracy:   72.3%
Optimized Accuracy:  99.0%
Accuracy Uplift:     +26.7%
------------------------------------------------------------
Total Tokens:        1,217,881
  - Prompt:          700,424
  - Completion:      517,457
Estimated Cost:      $1.5882
Time Taken:          1445.0s
Notes:               Uses gpt-4o for instruction generation. auto='light' preset.



---
## 6. GEPA

Evolutionary optimizer that uses reflection to evolve instructions based on feedback.

In [35]:
from dspy.teleprompt import GEPA

print("Running GEPA optimizer...")

start_time = time.time()
with dspy.track_usage() as usage:
    optimizer = GEPA(
        metric=exact_match_with_feedback,
        auto="light",
        reflection_lm=teacher_lm,
        num_threads=4,
        track_stats=True,
        use_merge=False,  # Disable merge for faster benchmarking
    )
    optimized_detector = optimizer.compile(
        AIDetector(),
        trainset=trainset,
        valset=valset,
    )
    
    # Evaluate
    optimized_accuracy = evaluate_optimized(optimized_detector, valset, exact_match)

elapsed = time.time() - start_time
total_usage = usage.get_total_tokens()

total_tokens = sum(d.get("total_tokens", 0) for d in total_usage.values())
prompt_tokens = sum(d.get("prompt_tokens", 0) for d in total_usage.values())
completion_tokens = sum(d.get("completion_tokens", 0) for d in total_usage.values())

result = BenchmarkResult(
    optimizer_name="GEPA",
    baseline_accuracy=baseline_accuracy,
    optimized_accuracy=optimized_accuracy,
    accuracy_uplift=optimized_accuracy - baseline_accuracy,
    total_tokens=total_tokens,
    prompt_tokens=prompt_tokens,
    completion_tokens=completion_tokens,
    cost_usd=calculate_cost(total_usage),
    time_seconds=elapsed,
    notes="Evolutionary optimizer with reflection. Uses gpt-4o for reflection.",
    usage_by_model=total_usage,
)

print_benchmark_result(result)
all_results.append(result)

2025/12/03 17:15:15 INFO dspy.teleprompt.gepa.gepa: Running GEPA for approx 784 metric calls of the program. This amounts to 3.88 full evals on the train+val set.
2025/12/03 17:15:15 INFO dspy.teleprompt.gepa.gepa: Using 101 examples for tracking Pareto scores. You can consider using a smaller sample of the valset to allow GEPA to explore more diverse solutions within the same budget. GEPA requires you to provide the smallest valset that is just large enough to match your downstream task distribution, while providing as large trainset as possible.


Running GEPA optimizer...


GEPA Optimization:   0%|          | 0/784 [00:00<?, ?rollouts/s]2025/12/03 17:18:10 INFO dspy.evaluate.evaluate: Average Metric: 69 / 101 (68.3%)
2025/12/03 17:18:10 INFO dspy.teleprompt.gepa.gepa: Iteration 0: Base program full valset score: 0.6831683168316832
GEPA Optimization:  13%|█▎        | 101/784 [02:54<19:42,  1.73s/rollouts]2025/12/03 17:18:10 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Selected program 0 score: 0.6831683168316832


Average Metric: 3.00 / 3 (100.0%): 100%|██████████| 3/3 [00:08<00:00,  2.84s/it]

2025/12/03 17:18:18 INFO dspy.evaluate.evaluate: Average Metric: 3 / 3 (100.0%)
2025/12/03 17:18:18 INFO dspy.teleprompt.gepa.gepa: Iteration 1: All subsample scores perfect. Skipping.
2025/12/03 17:18:18 INFO dspy.teleprompt.gepa.gepa: Iteration 1: Reflective mutation did not propose a new candidate
GEPA Optimization:  13%|█▎        | 104/784 [03:03<20:08,  1.78s/rollouts]2025/12/03 17:18:18 INFO dspy.teleprompt.gepa.gepa: Iteration 2: Selected program 0 score: 0.6831683168316832


...


Average Metric: 3.00 / 3 (100.0%): 100%|██████████| 3/3 [00:08<00:00,  2.75s/it]

2025/12/03 17:47:55 INFO dspy.evaluate.evaluate: Average Metric: 3 / 3 (100.0%)
2025/12/03 17:47:55 INFO dspy.teleprompt.gepa.gepa: Iteration 124: All subsample scores perfect. Skipping.
2025/12/03 17:47:55 INFO dspy.teleprompt.gepa.gepa: Iteration 124: Reflective mutation did not propose a new candidate
GEPA Optimization: 100%|█████████▉| 782/784 [32:40<00:05,  2.51s/rollouts]



Average Metric: 100.00 / 101 (99.0%): 100%|██████████| 101/101 [00:49<00:00,  2.03it/s]

2025/12/03 17:48:45 INFO dspy.evaluate.evaluate: Average Metric: 100 / 101 (99.0%)




OPTIMIZER: GEPA
Baseline Accuracy:   72.3%
Optimized Accuracy:  99.0%
Accuracy Uplift:     +26.7%
------------------------------------------------------------
Total Tokens:        983,276
  - Prompt:          598,859
  - Completion:      384,417
Estimated Cost:      $0.9765
Time Taken:          2010.1s
Notes:               Evolutionary optimizer with reflection. Uses gpt-4o for reflection.



---
## 7. SIMBA

Stochastic Introspective Mini-Batch Ascent - uses LLM to analyze performance and generate improvement rules.

In [36]:
from dspy.teleprompt import SIMBA

print("Running SIMBA optimizer...")

start_time = time.time()
with dspy.track_usage() as usage:
    optimizer = SIMBA(
        metric=exact_match,
        bsize=8,  # Mini-batch size (reduced for small dataset)
        num_candidates=4,  # Number of candidates per iteration
        max_steps=3,  # Number of optimization steps
        max_demos=4,
        prompt_model=teacher_lm,
        num_threads=4,
    )
    optimized_detector = optimizer.compile(
        AIDetector(),
        trainset=trainset,
        seed=42,
    )
    
    # Evaluate
    optimized_accuracy = evaluate_optimized(optimized_detector, valset, exact_match)

elapsed = time.time() - start_time
total_usage = usage.get_total_tokens()

total_tokens = sum(d.get("total_tokens", 0) for d in total_usage.values())
prompt_tokens = sum(d.get("prompt_tokens", 0) for d in total_usage.values())
completion_tokens = sum(d.get("completion_tokens", 0) for d in total_usage.values())

result = BenchmarkResult(
    optimizer_name="SIMBA",
    baseline_accuracy=baseline_accuracy,
    optimized_accuracy=optimized_accuracy,
    accuracy_uplift=optimized_accuracy - baseline_accuracy,
    total_tokens=total_tokens,
    prompt_tokens=prompt_tokens,
    completion_tokens=completion_tokens,
    cost_usd=calculate_cost(total_usage),
    time_seconds=elapsed,
    notes="Mini-batch ascent with self-reflection. Uses gpt-4o for rule generation.",
    usage_by_model=total_usage,
)

print_benchmark_result(result)
all_results.append(result)

2025/12/03 17:48:45 INFO dspy.teleprompt.simba: Starting batch 1 of 3.


Running SIMBA optimizer...


2025/12/03 17:48:47 INFO dspy.teleprompt.simba: Sampling program trajectories on 8 examples x 4 samples.


Processed 32 / 32 examples: 100%|██████████| 32/32 [00:57<00:00,  1.80s/it]

2025/12/03 17:49:44 INFO dspy.teleprompt.simba: Batch 1: Baseline mini-batch score: 0.8125

2025/12/03 17:49:44 INFO dspy.teleprompt.simba: Batch 1: Processing bucket #1, with max score 1.0, max-to-min gap 1.0, and max-to-avg gap 0.25.
2025/12/03 17:49:44 INFO dspy.teleprompt.simba: Batch 1: Invoking strategy: append_a_rule


...


Processed 404 / 404 examples: 100%|██████████| 404/404 [11:34<00:00,  1.72s/it]

2025/12/03 18:09:32 INFO dspy.teleprompt.simba: Final trainset scores: [0.6732673267326733, 0.9900990099009901, 1.0, 0.9702970297029703], Best: 1.0 (at index 2)






Average Metric: 100.00 / 101 (99.0%): 100%|██████████| 101/101 [00:46<00:00,  2.16it/s]

2025/12/03 18:10:19 INFO dspy.evaluate.evaluate: Average Metric: 100 / 101 (99.0%)




OPTIMIZER: SIMBA
Baseline Accuracy:   72.3%
Optimized Accuracy:  99.0%
Accuracy Uplift:     +26.7%
------------------------------------------------------------
Total Tokens:        607,784
  - Prompt:          307,312
  - Completion:      300,472
Estimated Cost:      $0.7337
Time Taken:          1293.7s
Notes:               Mini-batch ascent with self-reflection. Uses gpt-4o for rule generation.



---
## Fine-tuning Optimizers (Require Training Infrastructure)

The following optimizers require **actual model fine-tuning** (not just inference). They will NOT work with:
- LM Studio (inference only)
- Ollama (inference only)
- Standard OpenAI/Anthropic APIs

---

### Setup Instructions

#### Option A: BootstrapFinetune with SGLang (Local GPU)

**Installation:**
```bash
# SGLang for inference serving
pip install "sglang[all]>=0.4.4.post3" --find-links https://flashinfer.ai/whl/cu124/torch2.5/flashinfer-python

# Fine-tuning dependencies
pip install torch transformers==4.48.3 accelerate trl peft
```

**Usage:**
```python
from dspy.clients.lm_local import LocalProvider

# Initialize local model with SGLang
student_lm = dspy.LM(
    model="openai/local:Qwen/Qwen2.5-1.5B-Instruct",
    provider=LocalProvider(),
    max_tokens=2000
)
student_lm.launch()  # Starts SGLang server

# Configure BootstrapFinetune
optimizer = dspy.BootstrapFinetune(
    metric=your_metric,
    train_kwargs={
        "num_train_epochs": 5,
        "per_device_train_batch_size": 1,
        "gradient_accumulation_steps": 8,
        "learning_rate": 1e-5,
        "bf16": True,
    }
)
```

---

#### Option B: GRPO with Arbor (Recommended for RL Fine-tuning)

GRPO uses [Arbor](https://github.com/Ziems/arbor), a framework for optimizing DSPy programs with reinforcement learning.

**Installation:**
```bash
pip install arbor-ai

# Optional: Flash Attention for performance (may take 15+ min to install)
pip install flash-attn --no-build-isolation
```

**Usage:**
```python
import arbor
from arbor import ArborProvider
from dspy.teleprompt import GRPO
from dspy.clients.utils_finetune import MultiGPUConfig

# Start Arbor server
arbor_server_info = arbor.init()

# Connect DSPy to Arbor
provider = ArborProvider()
lm = dspy.LM(
    model="openai/arbor:Qwen/Qwen2.5-1.5B-Instruct",
    provider=provider,
    api_base=arbor_server_info["base_url"],
    api_key="arbor",
    temperature=1.0,
    max_tokens=2048,
)

# Configure GRPO
gpu_config = MultiGPUConfig(num_inference_gpus=1, num_training_gpus=1)

optimizer = GRPO(
    metric=your_metric,
    num_dspy_examples_per_grpo_step=4,
    num_rollouts_per_grpo_step=8,
    exclude_demos=True,  # Required
    multitask=True,      # Required
    num_train_steps=100,
    gpu_config=gpu_config,
    train_kwargs={
        "temperature": 0.9,
        "beta": 0.04,
        "learning_rate": 1e-6,
        "gradient_checkpointing": True,
    }
)
```

---

#### Hardware Requirements

| Optimizer | Min GPUs | Recommended | VRAM per GPU |
|-----------|----------|-------------|--------------|
| BootstrapFinetune | 1 | 1-2 | 16GB+ |
| GRPO | 2 | 4+ | 24GB+ (H100 ideal) |
| BetterTogether | 2 | 4+ | 24GB+ |

---

#### References

- [DSPy RL Multi-Hop Tutorial](https://dspy.ai/tutorials/rl_multihop/)
- [DSPy RL PAPILLON Tutorial](https://dspy.ai/tutorials/rl_papillon/)
- [Arbor GitHub](https://github.com/Ziems/arbor)
- [DSPy Classification Fine-tuning Tutorial](https://dspy.ai/tutorials/classification_finetuning/)

---

**If you don't have this infrastructure, skip these cells.** The prompt-based optimizers above (LabeledFewShot through SIMBA) work with any LLM provider and are typically sufficient for most use cases.

In [10]:
pip install "sglang[all]>=0.4.4.post3" --find-links https://flashinfer.ai/whl/cu124/torch2.5/flashinfer-python

Looking in links: https://flashinfer.ai/whl/cu124/torch2.5/flashinfer-python
  Using cached anthropic-0.75.0-py3-none-any.whl.metadata (28 kB)
INFO: pip is looking at multiple versions of sglang to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 8.1 MB/s  0:00:00 eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is still looking at multiple versions of sglang to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 11.8 MB/s  0:00:00 eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 12.9 MB/s  0:00:00 eta 0:00:01
  Installing build dependencies 

In [11]:
pip install torch transformers==4.48.3 accelerate trl peft

  Using cached tokenizers-0.21.4-cp39-abi3-macosx_11_0_arm64.whl.metadata (6.7 kB)
  Using cached datasets-4.4.1-py3-none-any.whl.metadata (19 kB)
INFO: pip is looking at multiple versions of trl to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of trl to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 31.6 MB/s  0:00:00 eta 0:00:01
Using cached tokenizers-0.21.4-cp39-abi3-macosx_11_0_arm64.whl (2.7 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 556.4/556.4 kB 11.9 MB/s  0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.1
   

In [17]:
# Check if fine-tuning infrastructure is available
import subprocess
import shutil

BOOTSTRAPFINETUNE_AVAILABLE = False
GRPO_AVAILABLE = False
LOCAL_MODEL = "Qwen/Qwen2.5-1.5B-Instruct"

# Check for GPU
try:
    import torch
    if torch.cuda.is_available():
        gpu_name = torch.cuda.get_device_name(0)
        gpu_count = torch.cuda.device_count()
        vram = torch.cuda.get_device_properties(0).total_memory / 1e9
        print(f"GPU available: {gpu_name}")
        print(f"GPU count: {gpu_count}")
        print(f"VRAM per GPU: {vram:.1f} GB")
        BOOTSTRAPFINETUNE_AVAILABLE = True
        if gpu_count >= 2:
            GRPO_AVAILABLE = True
    else:
        print("No GPU available.")
except ImportError:
    print("PyTorch not installed.")

# Check for Arbor (required for GRPO)
try:
    import arbor
    print("Arbor installed: Yes")
    if not GRPO_AVAILABLE:
        print("  (but need 2+ GPUs for GRPO)")
except ImportError:
    print("Arbor installed: No (run: pip install arbor-ai)")
    GRPO_AVAILABLE = False

# Check for SGLang (required for BootstrapFinetune)
try:
    import sglang
    print("SGLang installed: Yes")
except ImportError:
    print("SGLang installed: No (run: pip install 'sglang[all]')")
    BOOTSTRAPFINETUNE_AVAILABLE = False

print(f"\nLocal model: {LOCAL_MODEL}")
print(f"BootstrapFinetune available: {BOOTSTRAPFINETUNE_AVAILABLE}")
print(f"GRPO available: {GRPO_AVAILABLE}")

if not BOOTSTRAPFINETUNE_AVAILABLE and not GRPO_AVAILABLE:
    print("\nFine-tuning optimizers will be skipped. See setup instructions above.")

No GPU available.


/Users/hammermt/Codes/dspy-book/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
W1204 12:31:50.437000 73384 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


Arbor installed: No (run: pip install arbor-ai)
SGLang installed: No (run: pip install 'sglang[all]')

Local model: Qwen/Qwen2.5-1.5B-Instruct
BootstrapFinetune available: False
GRPO available: False

Fine-tuning optimizers will be skipped. See setup instructions above.


---
## 8. GRPO (Group Relative Policy Optimization)

Fine-tunes the model using reinforcement learning with group relative rewards.

In [ ]:
if GRPO_AVAILABLE:
    import arbor
    from arbor import ArborProvider
    from dspy.teleprompt import GRPO
    from dspy.clients.utils_finetune import MultiGPUConfig
    
    print("Running GRPO optimizer...")
    
    start_time = time.time()
    try:
        # Start Arbor server
        arbor_server_info = arbor.init()
        
        # Connect DSPy to Arbor with local model
        provider = ArborProvider()
        local_lm = dspy.LM(
            model=f"openai/arbor:{LOCAL_MODEL}",
            provider=provider,
            api_base=arbor_server_info["base_url"],
            api_key="arbor",
            temperature=1.0,
            max_tokens=2048,
        )
        
        # Create student with Arbor LM
        student = AIDetector()
        student.set_lm(local_lm)
        
        # Configure GPU allocation
        gpu_config = MultiGPUConfig(
            num_inference_gpus=1,
            num_training_gpus=1
        )
        
        optimizer = GRPO(
            metric=exact_match,
            num_threads=4,
            num_train_steps=10,  # Reduced for benchmarking
            num_dspy_examples_per_grpo_step=2,
            num_rollouts_per_grpo_step=4,
            exclude_demos=True,  # Required
            multitask=True,      # Required
            gpu_config=gpu_config,
            train_kwargs={
                "temperature": 0.9,
                "beta": 0.04,
                "learning_rate": 1e-6,
                "gradient_checkpointing": True,
            }
        )
        optimized_detector = optimizer.compile(
            student,
            trainset=trainset,
            valset=valset,
        )
        
        # Evaluate
        optimized_accuracy = evaluate_optimized(optimized_detector, valset, exact_match)
        
        elapsed = time.time() - start_time
        
        result = BenchmarkResult(
            optimizer_name="GRPO",
            baseline_accuracy=baseline_accuracy,
            optimized_accuracy=optimized_accuracy,
            accuracy_uplift=optimized_accuracy - baseline_accuracy,
            total_tokens=0,
            prompt_tokens=0,
            completion_tokens=0,
            cost_usd=0.0,  # Local model, no API cost
            time_seconds=elapsed,
            notes=f"RL fine-tuning with Arbor + {LOCAL_MODEL}. Local GPU compute.",
        )
        
        print_benchmark_result(result)
        all_results.append(result)
        
    except Exception as e:
        print(f"GRPO failed: {e}")
        result = BenchmarkResult(
            optimizer_name="GRPO",
            baseline_accuracy=baseline_accuracy,
            optimized_accuracy=0,
            accuracy_uplift=0,
            total_tokens=0,
            prompt_tokens=0,
            completion_tokens=0,
            cost_usd=0,
            time_seconds=time.time() - start_time,
            notes=f"FAILED: {str(e)[:100]}",
        )
        all_results.append(result)
else:
    print("Skipping GRPO - requires Arbor + 2 GPUs")
    result = BenchmarkResult(
        optimizer_name="GRPO",
        baseline_accuracy=baseline_accuracy,
        optimized_accuracy=0,
        accuracy_uplift=0,
        total_tokens=0,
        prompt_tokens=0,
        completion_tokens=0,
        cost_usd=0,
        time_seconds=0,
        notes="SKIPPED: Requires Arbor (pip install arbor-ai) + 2 GPUs",
    )
    all_results.append(result)

---
## 9. BootstrapFinetune

Bootstraps successful traces and uses them to fine-tune the model.

In [ ]:
if BOOTSTRAPFINETUNE_AVAILABLE:
    from dspy.teleprompt import BootstrapFinetune
    from dspy.clients.lm_local import LocalProvider
    
    print("Running BootstrapFinetune optimizer...")
    
    start_time = time.time()
    try:
        # Initialize local model with SGLang
        provider = LocalProvider()
        local_lm = dspy.LM(
            model=f"openai/local:{LOCAL_MODEL}",
            provider=provider,
            max_tokens=2000,
        )
        local_lm.launch()  # Start SGLang server
        
        # Create student with local LM
        student = AIDetector()
        student.set_lm(local_lm)
        
        # Use the OpenAI model as teacher for bootstrapping
        teacher = AIDetector()
        teacher.set_lm(teacher_lm)
        
        optimizer = BootstrapFinetune(
            metric=exact_match,
            multitask=True,
            num_threads=4,
            train_kwargs={
                "num_train_epochs": 3,
                "per_device_train_batch_size": 1,
                "gradient_accumulation_steps": 4,
                "learning_rate": 1e-5,
                "bf16": True,
            }
        )
        optimized_detector = optimizer.compile(
            student,
            teacher=teacher,
            trainset=trainset,
        )
        
        # Launch fine-tuned model for evaluation
        optimized_detector.get_lm().launch()
        
        # Evaluate
        optimized_accuracy = evaluate_optimized(optimized_detector, valset, exact_match)
        
        elapsed = time.time() - start_time
        
        result = BenchmarkResult(
            optimizer_name="BootstrapFinetune",
            baseline_accuracy=baseline_accuracy,
            optimized_accuracy=optimized_accuracy,
            accuracy_uplift=optimized_accuracy - baseline_accuracy,
            total_tokens=0,
            prompt_tokens=0,
            completion_tokens=0,
            cost_usd=0.0,
            time_seconds=elapsed,
            notes=f"SGLang + {LOCAL_MODEL}. Teacher: gpt-5.",
        )
        
        print_benchmark_result(result)
        all_results.append(result)
        
    except Exception as e:
        print(f"BootstrapFinetune failed: {e}")
        result = BenchmarkResult(
            optimizer_name="BootstrapFinetune",
            baseline_accuracy=baseline_accuracy,
            optimized_accuracy=0,
            accuracy_uplift=0,
            total_tokens=0,
            prompt_tokens=0,
            completion_tokens=0,
            cost_usd=0,
            time_seconds=time.time() - start_time,
            notes=f"FAILED: {str(e)[:100]}",
        )
        all_results.append(result)
else:
    print("Skipping BootstrapFinetune - requires SGLang + GPU")
    result = BenchmarkResult(
        optimizer_name="BootstrapFinetune",
        baseline_accuracy=baseline_accuracy,
        optimized_accuracy=0,
        accuracy_uplift=0,
        total_tokens=0,
        prompt_tokens=0,
        completion_tokens=0,
        cost_usd=0,
        time_seconds=0,
        notes="SKIPPED: Requires SGLang (pip install 'sglang[all]') + GPU",
    )
    all_results.append(result)

---
## 10. BetterTogether

Combines prompt optimization and weight optimization in an alternating strategy.

In [ ]:
if BOOTSTRAPFINETUNE_AVAILABLE:
    from dspy.teleprompt import BetterTogether
    from dspy.clients.lm_local import LocalProvider
    
    print("Running BetterTogether optimizer...")
    print("Note: BetterTogether is experimental and requires dspy.settings.experimental = True")
    
    start_time = time.time()
    try:
        # Initialize local model with SGLang
        provider = LocalProvider()
        local_lm = dspy.LM(
            model=f"openai/local:{LOCAL_MODEL}",
            provider=provider,
            max_tokens=2000,
        )
        local_lm.launch()
        
        # Create student with local LM
        student = AIDetector()
        student.set_lm(local_lm)
        
        optimizer = BetterTogether(
            metric=exact_match,
            seed=42,
        )
        optimized_detector = optimizer.compile(
            student,
            trainset=trainset,
            strategy="p -> w",  # Prompt optimization then weight optimization
        )
        
        # Evaluate
        optimized_accuracy = evaluate_optimized(optimized_detector, valset, exact_match)
        
        elapsed = time.time() - start_time
        
        result = BenchmarkResult(
            optimizer_name="BetterTogether",
            baseline_accuracy=baseline_accuracy,
            optimized_accuracy=optimized_accuracy,
            accuracy_uplift=optimized_accuracy - baseline_accuracy,
            total_tokens=0,
            prompt_tokens=0,
            completion_tokens=0,
            cost_usd=0.0,
            time_seconds=elapsed,
            notes=f"SGLang + {LOCAL_MODEL}. Strategy: p -> w. Experimental.",
        )
        
        print_benchmark_result(result)
        all_results.append(result)
        
    except Exception as e:
        print(f"BetterTogether failed: {e}")
        result = BenchmarkResult(
            optimizer_name="BetterTogether",
            baseline_accuracy=baseline_accuracy,
            optimized_accuracy=0,
            accuracy_uplift=0,
            total_tokens=0,
            prompt_tokens=0,
            completion_tokens=0,
            cost_usd=0,
            time_seconds=time.time() - start_time,
            notes=f"FAILED: {str(e)[:100]}",
        )
        all_results.append(result)
else:
    print("Skipping BetterTogether - requires SGLang + GPU")
    result = BenchmarkResult(
        optimizer_name="BetterTogether",
        baseline_accuracy=baseline_accuracy,
        optimized_accuracy=0,
        accuracy_uplift=0,
        total_tokens=0,
        prompt_tokens=0,
        completion_tokens=0,
        cost_usd=0,
        time_seconds=0,
        notes="SKIPPED: Requires SGLang + GPU. Experimental.",
    )
    all_results.append(result)

---
## Summary: Optimizer Comparison

Final comparison of all optimizers across all metrics.

In [ ]:
# Create summary DataFrame
summary_data = []
for r in all_results:
    summary_data.append({
        "Optimizer": r.optimizer_name,
        "Baseline Acc": f"{r.baseline_accuracy*100:.1f}%",
        "Optimized Acc": f"{r.optimized_accuracy*100:.1f}%" if r.optimized_accuracy > 0 else "N/A",
        "Uplift": f"{r.accuracy_uplift*100:+.1f}%" if r.optimized_accuracy > 0 else "N/A",
        "Time (s)": f"{r.time_seconds:.1f}" if r.time_seconds > 0 else "N/A",
        "Notes": r.notes[:50] + "..." if len(r.notes) > 50 else r.notes,
    })

summary_df = pd.DataFrame(summary_data)

print("\n" + "="*100)
print("OPTIMIZER BENCHMARK SUMMARY")
print("="*100)
print(f"\nBaseline Model: openai/gpt-5-mini")
print(f"Teacher/Reflection Model: openai/gpt-5")
print(f"Fine-tuning Model: {LOCAL_MODEL}")
print(f"\nTraining Examples: {len(trainset)}")
print(f"Validation Examples: {len(valset)}")
print("\n")

# Display with pandas
pd.set_option('display.max_colwidth', 60)
print(summary_df.to_string(index=False))

# Find best optimizer
valid_results = [r for r in all_results if r.optimized_accuracy > 0]
if valid_results:
    best_by_accuracy = max(valid_results, key=lambda r: r.accuracy_uplift)
    
    print("\n" + "-"*100)
    print(f"Best by Accuracy Uplift: {best_by_accuracy.optimizer_name} ({best_by_accuracy.accuracy_uplift*100:+.1f}%)")
    print("-"*100)

In [ ]:
# Detailed token usage breakdown
print("\n" + "="*80)
print("DETAILED TOKEN USAGE BY MODEL")
print("="*80)

for r in all_results:
    if r.usage_by_model:
        print(f"\n{r.optimizer_name}:")
        for model, usage in r.usage_by_model.items():
            prompt = usage.get('prompt_tokens', 0)
            completion = usage.get('completion_tokens', 0)
            total = usage.get('total_tokens', prompt + completion)
            print(f"  {model}:")
            print(f"    Prompt tokens:     {prompt:,}")
            print(f"    Completion tokens: {completion:,}")
            print(f"    Total tokens:      {total:,}")

In [ ]:
# Save results to CSV
results_df = pd.DataFrame([{
    "optimizer": r.optimizer_name,
    "baseline_accuracy": r.baseline_accuracy,
    "optimized_accuracy": r.optimized_accuracy,
    "accuracy_uplift": r.accuracy_uplift,
    "total_tokens": r.total_tokens,
    "prompt_tokens": r.prompt_tokens,
    "completion_tokens": r.completion_tokens,
    "cost_usd": r.cost_usd,
    "time_seconds": r.time_seconds,
    "notes": r.notes,
} for r in all_results])

results_df.to_csv("optimizer_benchmark_results.csv", index=False)
print("Results saved to optimizer_benchmark_results.csv")